In [1]:
###############################################################################
#################################################################################
####################################################################################
#######################################################################################

In [2]:
"""This notebook is design to train and evaluate an AI based on llama3.2 architecture and fine tuned
for chat and Text generation of potato varieties and their descriptions. We train Llama3 for causalModel generation
while the MPNet sentence transformers for  will be fine tuned sor scoring or rather sequence classification"""
# Install dependencies
!pip install -U transformers datasets accelerate peft bitsandbytes wandb flash-attn
!pip install trl==0.12.2

import os
import logging
import torch
import pandas as pd
import bitsandbytes as bnb
#from bitsandbytes import BitsAndBytesConfig
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer
from huggingface_hub import login
import wandb
#print(trl.__version__)

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Hugging Face and Weights & Biases login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACEHUB_API_TOKEN")
login(token=hf_token)

wb_token = user_secrets.get_secret("wandb")
wandb.login(key=wb_token)

# Initialize WandB
run = wandb.init(
    project="Llama Fine-Tune Potato Wizard",
    job_type="training",
    anonymous="allow"
)

# Load datasets
ReportSDGDatapath = '/kaggle/input/futproofagricaireportsvssdgtargets/curated_agric_dataset_v5.csv'
InputResponseDatapath = '/kaggle/input/futproofagricaireportsvssdgtargets/QA_data.csv'

try:
    ReportSDGData = pd.read_csv(ReportSDGDatapath)
    InputResponseData = pd.read_csv(InputResponseDatapath)
    logger.info("Datasets loaded successfully.")
except Exception as e:
    logger.error(f"Error loading datasets: {e}")
    raise

# Preprocess datasets
InputResponseData.rename(columns={'response': 'premise', 'input': 'instructions'}, inplace=True)

InputResponseData = InputResponseData[['instructions', 'premise']]
#ReportSDGData['instructions'] =  "This is a general information on potato traits, varieties, cultivation, socio-economics, genetics, and breeding amid climate change and disease prevalence."
ReportSDGData = ReportSDGData[['instructions', 'premise']]

#ReportSDGDataCombined.dropna(subset=['hypothesis'], inplace=True)
#ReportSDGDataCombined['instructions'] =  "General information on potato traits, cultivation, socio-economics, genetics, and breeding amid climate change and disease prevalence."

# Activate this code when we get the resouces to train the large dataset
#ReportSDGDataCombined = pd.concat([ReportSDGData, InputResponseData], axis=0)

#Then Deactivate this code
rows_to_drop = list(range(5394, 9284)) + list(range(9662, 10211)) + list(range(10581, 10898)) + list(range(11146, 11261)) + list(range(12307, 13635))
ReportSDGDataCombined = ReportSDGData.drop(rows_to_drop)

# Save combined data to JSON
json_file = "data.json"
try:
    ReportSDGDataCombined.to_json(json_file, orient="records", indent=4)
    logger.info(f"Data saved to {json_file}.")
except Exception as e:
    logger.error(f"Error saving dataset: {e}")
    raise

# Load dataset for training
try:
    dataset = load_dataset("json", data_files=json_file, split="train")
    logger.info("Dataset loaded successfully into Hugging Face format.")
except Exception as e:
    logger.error(f"Error loading dataset into Hugging Face format: {e}")
    raise

compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)
#quant_config

# Load model and tokenizer
base_model = "/kaggle/input/llama-3.2/transformers/1b-instruct/1"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    #return_dict=True,
    #torch_dtype=torch.float16,
    quantization_config=quant_config,
    device_map={"": 0},
    #device_map="auto",
    trust_remote_code=True
)

model.config.use_chache = False
model.config.pretraining_tp = 1

# Ensure tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

# Format the dataset for training
instruction = "You are a top-rated plant breeder and agronomy service agent named Shadrack. Give optimum potato trait combinations for potato varieties amid climate change and disease pressure. Be polite to farmers and breeders and answer all their questions."

def format_chat_template(row):
    row_json = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": row["instructions"]},
        {"role": "assistant", "content": row["premise"]}
    ]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(format_chat_template)


# Function to debug and inspect model architecture
def debug_model_architecture(model):
    for name, module in model.named_modules():
        print(name, type(module))

# Uncomment the following line to inspect the model architecture
# debug_model_architecture(model)

# LoRA-compatible modules
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit  # This works for 4-bit quantized models
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # Exclude `lm_head` if it's listed
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)
if not modules:
    logger.warning(
        "No compatible modules found for LoRA injection. Using default settings."
    )
    # Optionally fallback to standard layers
    modules = ["q_proj", "v_proj"
               #, "k_proj", "o_proj"
              ]

logger.info(f"LoRA modules identified: {modules}")

# Adjust LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=modules,  # Dynamically identified modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

try:
    model = get_peft_model(model, lora_config)
    logger.info("LoRA modules successfully injected into the model.")
except ValueError as e:
    logger.error(f"Failed to inject LoRA modules: {e}")
    raise





model = get_peft_model(model, lora_config)

# Training arguments
training_args = TrainingArguments(
    output_dir="./llama-finetunes-potato-wizard_v59",
    per_device_train_batch_size=4,
    #per_device_eval_batch_size=2,
    optim = "adafactor",
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    weight_decay=1e-2,
    #group_by_length = True,
    num_train_epochs=1,
    #max_steps=1,
    logging_dir="./logs",
    save_strategy="epoch",
    evaluation_strategy="steps",
    logging_steps=10,
    fp16=True,
    bf16=False,
    report_to="wandb"
)

# Split dataset into train and validation sets
train_validation_split = dataset.train_test_split(test_size=0.2, seed=42)
tokenized_dataset = DatasetDict({
    "train": train_validation_split["train"],
    "validation": train_validation_split["test"]
})

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    peft_config=lora_config,
    max_seq_length=512, # we realized that the SFTTrainer always pads by default the sequences to the max_seq_length argument of the 
    #SFTTrainer. If none is passed, the trainer will retrieve that value from the tokenizer. Some tokenizers do not provide a default 
    #value, so there is a check to retrieve the minimum between 2048 and that value.
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_args
    #packing=False
)

# Train and save the model
trainer.train()

# Save model's state_dict to pytorch_model.bin explicitly
torch.save(model.state_dict(), './llama-finetunes-potato-wizard_v59/pytorch_model.bin')
trainer.model.save_pretrained("./llama-finetunes-potato-wizard_v59")
tokenizer.save_pretrained("./llama-finetunes-potato-wizard_v59")
model.config.save_pretrained("/llama-finetunes-potato-wizard_v59")

# Push the fine-tuned model to Hugging Face Hub
repo_name = "llama-potato-wizard_v59"
trainer.model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

#save the model in gg format to be used with ollama locally
#model.save_pretrained_gguf("./llama-finetuned-potato-wizard-gguf", tokenizer, quantization_method = "f16")


# End the WandB run
wandb.finish()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.7 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.7.2.post1-cp310-cp310-linux_x86_64.whl size=190150491 sha256=801fc83166b3ca827813132b8a687d1c81cde0cd44353a018

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shadrack-imai (shadrack-imai-kenya-agricultural-and-livestock-research-). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241230_103729-6u7tkw7u
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run brisk-eon-88
wandb: ⭐️ View project at https://wandb.ai/shadrack-imai-kenya-agricultural-and-livestock-research-/Llama%20Fine-Tune%20Potato%20Wizard
wandb: 🚀 View run at https://wandb.ai/shadrack-imai-kenya-agricultural-and-livestock-research-/Llama

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/19564 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/peft/mapping.py:185: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from '/kaggle/input/llama-3.2/transformers/1b-instruct/1' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluatio

...

Step,Training Loss,Validation Loss
10,2.544900,2.008823
20,1.875500,1.860990
30,1.752500,1.795704
40,1.726100,1.719487
50,1.747200,1.686727
60,1.596200,1.671634
70,1.722400,1.653763
80,1.692000,1.640997
90,1.613600,1.634123
100,1.616200,1.625907


adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:               eval/loss █▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:            eval/runtime ▂▁▂▁▂▃▄▅▆▅▄▄▃▅▅▅▇▆▆▆▇▆▇▇▇▇▆▆▇▆▇█▇▇█▇▆▇▇▇
wandb: eval/samples_per_second █████▆▆▄▃▄▆▄▆▄▄▄▃▃▃▃▁▃▃▃▁▁▃▃▁▃▁▁▃▃▁▁▃▃▃▁
wandb:   eval/steps_per_second ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:             train/epoch ▁▁▁▁▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
wandb:       train/global_step ▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
wandb:         train/grad_norm █▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▂▁▂▁▁▁▁▁▂▁▂▁▁▁▁▁▁
wandb:     train/learning_rate ████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
wandb:              train/loss █▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▂▂▁▂▂▂▂▂▂▃▂▂▁▂▁▂▂
wandb: 
wandb: Run summary:
wandb:                eval/loss 1.52829
wandb:             eval/runtime 757.6727
wandb:  eval/samples_per_second 5.164
wandb:    eval/steps_per_second 0.647
wandb:               total_flos 4.026025940085965e+1